# <p align="center"><b style="color:#1f6feb;">📄🤖 Intelligent Question Answering System over PDF using LLMs (Hugging Face) 🤖📄</b></p>

In [1]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 57.6 MB/s eta 0:00:00


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import fitz  # PyMuPDF
import torch
import textwrap
# https://huggingface.co/google/flan-t5-base
# Load FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ---------------------- PDF Reader ----------------------
def read_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# ---------------------- Chunk Splitter ----------------------
def split_into_chunks(text, max_words=50):
    words = text.split()
    return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
# ---------------------- QA Function ----------------------
def ask_question_flan(question, context_chunks):
    best_answer = ""
    best_score = float("-inf")

    for chunk in context_chunks:
        prompt = f"answer the question based on the context:\ncontext: {chunk}\nquestion: {question}"
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to(device)

        outputs = model.generate(input_ids, max_length=100)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Simple heuristic: prefer longer answers with more content
        score = len(answer.strip().split())
        if score > best_score:
            best_answer = answer
            best_score = score

    return best_answer.strip()

In [4]:
pdf_path = "/content/Cairo_Info.pdf"
print(" Reading PDF...")
text = read_pdf_text(pdf_path)

print(" Splitting text...")
chunks = split_into_chunks(text,20)

question = input(" Ask your question: ")
print("🤖 Thinking...")

answer = ask_question_flan(question, chunks)

print("\n✅ Answer:")
print(textwrap.fill(answer, width=80))

 Reading PDF...
 Splitting text...
 Ask your question: what is the cairo
🤖 Thinking...

✅ Answer:
renowned for its ancient landmarks, including the nearby Giza Pyramids and the
Great Sphinx, as well as its Islamic architecture,
